<a href="https://colab.research.google.com/github/ravi-prakash1907/Machine-Learning-for-Cyber-Security/blob/main/frombeg/smsClassifactio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prepare the dataset

#### Tokanize and all


In [1]:
## libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import requests
import re

In [2]:
## location to store the dataset
try:
  os.mkdir('Datasets')
  dataFile = 'Datasets/SMSSpamCollection.txt'
except:
  dataFile = 'Datasets/SMSSpamCollection.txt'

In [3]:
## fun. to download the data from any url
def downloadData(fileURL, saveAs='downloaded'):
  req = requests.get(fileURL)
  fileURLContent = req.content
  data = open(saveAs, 'wb')
  
  data.write(fileURLContent)
  data.close()

In [34]:
## location of the file
fileURL = 'https://raw.githubusercontent.com/ravi-prakash1907/Machine-Learning-for-Cyber-Security/main/Datasets/smsspamcollection/SMSSpamCollection.txt?token=AJGAAOA2SOYVM324GI45BLTATFASY'

## downloading the from url
downloadData(fileURL,dataFile)

In [35]:
## importing data
sms_spam = pd.read_csv(dataFile, sep='\t', header=None, names=['Label', 'SMS'])

print("Shape",sms_spam.shape)
sms_spam.head()

Shape (5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [36]:
## checking for ham/spam SMS ratio from the whole bulk
sms_spam['Label'].value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: Label, dtype: float64

## Data Cleaning

In [37]:
def cleanDF(df):
  ### cleaning
  df['SMS'] = df['SMS'].str.lower()
  #df.head(3)

  ## removing the blank spaces
  df['SMS'] = df['SMS'].str.strip()
  #df.head(3)

  ## removing punctuations
  def removePunctuations(gotStr):
    cleanedStr = re.sub(r'[^\w\s]', '', gotStr)
    return cleanedStr
  
  df['SMS'] = df['SMS'].apply(removePunctuations)
  #df.head()

  return df

In [38]:
## cleaning using function call
sms_spam = cleanDF(sms_spam)
sms_spam.head()

,Label,SMS
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i dont think he goes to usf he lives aroun...


### Data Preparation

In [39]:
## introducing dummy variable
numLab = [int(s == 'spam') for s in sms_spam['Label']]
sms_spam['label_num'] = numLab
sms_spam.head()

,Label,SMS,label_num
0,ham,go until jurong point crazy available only in ...,0
1,ham,ok lar joking wif u oni,0
2,spam,free entry in 2 a wkly comp to win fa cup fina...,1
3,ham,u dun say so early hor u c already then say,0
4,ham,nah i dont think he goes to usf he lives aroun...,0


In [40]:
## changing to lower case
sms_spam['SMS'] = sms_spam['SMS'].str.split()

vocabulary = []
for sms in sms_spam['SMS']:
   for word in sms:
      vocabulary.append(word)

vocabulary = list(set(vocabulary))

In [67]:
word_counts_per_sms = {unique_word: [0] * len(sms_spam['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(sms_spam['SMS']):
  for word in sms:
    word_counts_per_sms[word][index] += 1

In [68]:
word_counts = pd.DataFrame(word_counts_per_sms)
for col in word_counts.columns[:-1]:
  word_counts[col] %= 2

word_counts.head()

,minus,100psms,twins,5terror,obedient,submitted,karaoke,men,moral,weak,shame,hme,boost,bedreal,icon,p399,ltdhelpdesk,pure,hypotheticalhuagauahahuagahyuhagga,particular,choice,laptop,yeovil,waheed,07xxxxxxxxx,4my,pubs,jsut,08718727868,alert,toothpaste,freaking,bennys,cave,tyler,problemfree,wwwmovietriviatv,chillaxin,santa,freshers,...,best,portions,receiving,outsomewhere,arsenal,dont4get2text,spoons,maths,jenny,eight,chapter,textin,knock,wwwbridalpetticoatdreamscouk,applespairsall,ummmawill,pobox75ldns7,telphone,raping,32f,avoiding,3mins,zindgi,difference,post,bag,stuffed,noe,uxxxx,prestige,0870141701216,selling,later,memorable,printing,lifeyou,83332please,commit,meat,9153
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [69]:
word_counts['label'] = sms_spam['label_num']
word_counts.head()

,minus,100psms,twins,5terror,obedient,submitted,karaoke,men,moral,weak,shame,hme,boost,bedreal,icon,p399,ltdhelpdesk,pure,hypotheticalhuagauahahuagahyuhagga,particular,choice,laptop,yeovil,waheed,07xxxxxxxxx,4my,pubs,jsut,08718727868,alert,toothpaste,freaking,bennys,cave,tyler,problemfree,wwwmovietriviatv,chillaxin,santa,freshers,...,portions,receiving,outsomewhere,arsenal,dont4get2text,spoons,maths,jenny,eight,chapter,textin,knock,wwwbridalpetticoatdreamscouk,applespairsall,ummmawill,pobox75ldns7,telphone,raping,32f,avoiding,3mins,zindgi,difference,post,bag,stuffed,noe,uxxxx,prestige,0870141701216,selling,later,memorable,printing,lifeyou,83332please,commit,meat,9153,label
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [70]:
word_counts.shape

(5572, 9586)

In [71]:
word_counts.drop_duplicates()
word_counts.head()

,minus,100psms,twins,5terror,obedient,submitted,karaoke,men,moral,weak,shame,hme,boost,bedreal,icon,p399,ltdhelpdesk,pure,hypotheticalhuagauahahuagahyuhagga,particular,choice,laptop,yeovil,waheed,07xxxxxxxxx,4my,pubs,jsut,08718727868,alert,toothpaste,freaking,bennys,cave,tyler,problemfree,wwwmovietriviatv,chillaxin,santa,freshers,...,portions,receiving,outsomewhere,arsenal,dont4get2text,spoons,maths,jenny,eight,chapter,textin,knock,wwwbridalpetticoatdreamscouk,applespairsall,ummmawill,pobox75ldns7,telphone,raping,32f,avoiding,3mins,zindgi,difference,post,bag,stuffed,noe,uxxxx,prestige,0870141701216,selling,later,memorable,printing,lifeyou,83332please,commit,meat,9153,label
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [72]:
word_counts.shape

(5572, 9586)

---  
---

## Naive Bayes

In [73]:
df = word_counts.copy()
df.head()

,minus,100psms,twins,5terror,obedient,submitted,karaoke,men,moral,weak,shame,hme,boost,bedreal,icon,p399,ltdhelpdesk,pure,hypotheticalhuagauahahuagahyuhagga,particular,choice,laptop,yeovil,waheed,07xxxxxxxxx,4my,pubs,jsut,08718727868,alert,toothpaste,freaking,bennys,cave,tyler,problemfree,wwwmovietriviatv,chillaxin,santa,freshers,...,portions,receiving,outsomewhere,arsenal,dont4get2text,spoons,maths,jenny,eight,chapter,textin,knock,wwwbridalpetticoatdreamscouk,applespairsall,ummmawill,pobox75ldns7,telphone,raping,32f,avoiding,3mins,zindgi,difference,post,bag,stuffed,noe,uxxxx,prestige,0870141701216,selling,later,memorable,printing,lifeyou,83332please,commit,meat,9153,label
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [74]:
df.describe()

,minus,100psms,twins,5terror,obedient,submitted,karaoke,men,moral,weak,shame,hme,boost,bedreal,icon,p399,ltdhelpdesk,pure,hypotheticalhuagauahahuagahyuhagga,particular,choice,laptop,yeovil,waheed,07xxxxxxxxx,4my,pubs,jsut,08718727868,alert,toothpaste,freaking,bennys,cave,tyler,problemfree,wwwmovietriviatv,chillaxin,santa,freshers,...,portions,receiving,outsomewhere,arsenal,dont4get2text,spoons,maths,jenny,eight,chapter,textin,knock,wwwbridalpetticoatdreamscouk,applespairsall,ummmawill,pobox75ldns7,telphone,raping,32f,avoiding,3mins,zindgi,difference,post,bag,stuffed,noe,uxxxx,prestige,0870141701216,selling,later,memorable,printing,lifeyou,83332please,commit,meat,9153,label
count,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.0,5572.000000,...,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.0,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000,5572.000000
mean,0.000179,0.000179,0.000179,0.000179,0.000179,0.000179,0.000538,0.001615,0.000897,0.000538,0.000718,0.000179,0.000179,0.000179,0.000179,0.000359,0.000179,0.000179,0.000179,0.000179,0.000897,0.002154,0.000179,0.000359,0.000538,0.000179,0.000179,0.000359,0.000359,0.000718,0.000179,0.000179,0.000179,0.000179,0.000897,0.000179,0.000359,0.000179,0.0,0.000179,...,0.000179,0.000718,0.000179,0.000179,0.000179,0.000179,0.000179,0.000359,0.000538,0.000179,0.000179,0.0,0.000179,0.000179,0.000179,0.000179,0.000538,0.000179,0.000179,0.000359,0.000718,0.000179,0.000359,0.002872,0.000897,0.000179,0.003589,0.000179,0.000179,0.000179,0.000538,0.022972,0.000179,0.000179,0.000179,0.000179,0.000179,0.000179,0.000179,0.134063
std,0.013397,0.013397,0.013397,0.013397,0.013397,0.013397,0.023199,0.040161,0.029945,0.023199,0.026786,0.013397,0.013397,0.013397,0.013397,0.018944,0.013397,0.013397,0.013397,0.013397,0.029945,0.046361,0.013397,0.018944,0.023199,0.013397,0.013397,0.018944,0.018944,0.026786,0.013397,0.013397,0.013397,0.013397,0.029945,0.013397,0.018944,0.013397,0.0,0.013397,...,0.013397,0.026786,0.013397,0.013397,0.013397,0.013397,0.013397,0.018944,0.023199,0.013397,0.013397,0.0,0.013397,0.013397,0.013397,0.013397,0.023199,0.013397,0.013397,0.018944,0.026786,0.013397,0.018944,0.053514,0.029945,0.013397,0.059809,0.013397,0.013397,0.013397,0.023199,0.149828,0.013397,0.013397,0.013397,0.013397,0.013397,0.013397,0.013397,0.340751
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0

### Data Praparation - 2

In [75]:
def groupCol(dataCol):
  dataCol = pd.Series(dataCol)
  return [str(val) for val in dataCol//10]

In [76]:
def prepare(df):
  ## grouping all cols
  for col in tuple(df.columns[:-1]):
    df[col] = groupCol(dataCol = df[col])

### Naive Bayes' Classifier
_It takes the tuples for set of the:_  
1. Dependent variables  
2. Values of any datapoint
3. Name of cloumn i.e. to be predicted

In [77]:
## gives rows with given set of cols
def getOneLabel(df,label,predLabel):
  tempDf = df.copy()
  x = [df.loc[i][label] in predLabel for i in df.index]
  count = 0
  ind = []
  for val in x:
    if not val:
      ind.append(count)
    count += 1
  
  #print(ind)
  tempDf.drop(ind, inplace=True)
  prepare(tempDf)
  return tempDf

In [78]:
## classifier
def naiveBayesianPredictor(df, labels, given, predCol):
  possibleCases = list(df[predCol].unique())
  finalPredDescription = {}
  
  for possibleCase in possibleCases:
    tempDF = getOneLabel(df, predCol, [possibleCase])
    preds = len(tempDF) / len(df)


    for index in range(len(given)):
      partialProb = tempDF[labels[index]].value_counts() / len(tempDF)

      if given[index] not in np.array(partialProb.index, dtype='uint64'):    
        tempList = list(np.array(partialProb.index, dtype='uint64'))
        largers = [val for val in tempList if val>int(given[index])]
        if largers and int(given[index])/largers[0] >= 0.5:
          preds *= partialProb[str(largers[0])]
        else:
          continue
      else:
        preds *= partialProb[given[index]]  #Naive bayesian classifier
    
    finalPredDescription[possibleCase] = preds
  
  ## converting predictions to pandas Series
  finalPredDescription = pd.Series(finalPredDescription)
  ## choosing MAP (Maximum A Posteriori)
  if finalPredDescription[0] == finalPredDescription[1]:
    finalPred = -1
  else:
    finalPred = str(finalPredDescription[finalPredDescription == max(finalPredDescription)].index[0])

  return finalPred,finalPredDescription

In [79]:
def predict(df,labels, given, predCol, describe = False):
  given = groupCol(dataCol=given)
  
  finalPrediction = naiveBayesianPredictor(df, labels, given, predVar)
  
  if finalPrediction == -1:
    print("Tie for the given e-mail!! Get a human to check for spam!")
  else:
    res = ' ' if bool(int(finalPrediction[0])) else ' NOT '
    print("\nGiven e-mail is{}Spam! \nLabel Value: {}".format(res,finalPrediction[0]))
  if describe:
    print("\nHere 1 indicates the probability of being a Spam E-Mail!")
    print("Probability Table:")
    return pd.DataFrame([finalPrediction[1]])


### Train-Test Split

In [80]:
# Randomize the dataset
data_randomized = df.sample(frac=1, random_state=1)

# Calculate index for split
training_test_index = round(len(data_randomized) * 0.8)

# Split into training and test sets
training_set = data_randomized[:training_test_index].reset_index(drop=True)
test_set = data_randomized[training_test_index:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(4458, 9586)
(1114, 9586)


In [82]:
training_set['label'].value_counts(normalize=True)

0    0.86541
1    0.13459
Name: label, dtype: float64

In [83]:
test_set['label'].value_counts(normalize=True)

0    0.868043
1    0.131957
Name: label, dtype: float64

In [84]:
# After splitting
training_set.head(3)

,minus,100psms,twins,5terror,obedient,submitted,karaoke,men,moral,weak,shame,hme,boost,bedreal,icon,p399,ltdhelpdesk,pure,hypotheticalhuagauahahuagahyuhagga,particular,choice,laptop,yeovil,waheed,07xxxxxxxxx,4my,pubs,jsut,08718727868,alert,toothpaste,freaking,bennys,cave,tyler,problemfree,wwwmovietriviatv,chillaxin,santa,freshers,...,portions,receiving,outsomewhere,arsenal,dont4get2text,spoons,maths,jenny,eight,chapter,textin,knock,wwwbridalpetticoatdreamscouk,applespairsall,ummmawill,pobox75ldns7,telphone,raping,32f,avoiding,3mins,zindgi,difference,post,bag,stuffed,noe,uxxxx,prestige,0870141701216,selling,later,memorable,printing,lifeyou,83332please,commit,meat,9153,label
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [86]:
temp = [x == 0 for x in test_set['label']]
temp[11:20]

[True, True, True, True, True, True, True, True, False]

---  
---

### Predictions

In [87]:
def realVal(intVal):
  if intVal:
    print("Actually Spam!")
  else:
    print("Actually Ham!")

In [88]:
labels = tuple(df.columns[:-1])
predVar = df.columns[-1]

#### Test case

In [89]:
## case 1
given = list(test_set.iloc[0])
popped = given.pop()

realVal(popped)

Actually Ham!


In [90]:
predict(training_set, labels, given, predVar, describe=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  



Given e-mail is NOT Spam! 
Label Value: 0

Here 1 indicates the probability of being a Spam E-Mail!
Probability Table:


,0,1
0,0.86541,0.13459


In [91]:
## case 2
given = list(test_set.iloc[19])
popped = given.pop()

realVal(popped)

Actually Spam!


In [92]:
predict(training_set, labels, given, predVar, describe=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  



Given e-mail is NOT Spam! 
Label Value: 0

Here 1 indicates the probability of being a Spam E-Mail!
Probability Table:


,0,1
0,0.86541,0.13459


---  
---

## Using `sklearn`

In [93]:
#Import Gaussian Naive Bayes model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import sklearn.model_selection as model_selection

In [94]:
df = pd.read_csv(dataFile, sep='\t', header=None, names=['Label', 'SMS'])

df = cleanDF(df)
numLab = [int(s == 'spam') for s in df['Label']]
df['label_num'] = numLab
df.head()

,Label,SMS,label_num
0,ham,go until jurong point crazy available only in ...,0
1,ham,ok lar joking wif u oni,0
2,spam,free entry in 2 a wkly comp to win fa cup fina...,1
3,ham,u dun say so early hor u c already then say,0
4,ham,nah i dont think he goes to usf he lives aroun...,0


In [95]:
X,y = df.SMS,df.label_num
X_train,X_test,y_train,y_test = model_selection.train_test_split(X, y, test_size=0.30)

In [96]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(X_train.values)

In [97]:
classifier = MultinomialNB()
targets = y_train.values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [98]:
examples = df['SMS'][0:10]

example_count = vectorizer.transform(examples)
predictions = classifier.predict(example_count)

In [99]:
print("Predicted: {} \nActual:    {}".format(list(predictions),list(df['label_num'][0:10])))

Predicted: [0, 0, 1, 0, 0, 0, 0, 0, 1, 1] 
Actual:    [0, 0, 1, 0, 0, 1, 0, 0, 1, 1]
